In [1]:
'''
Objective:
•
Utilize multilinear regression on historical data collected by an analytics company to identify factors affecting the profitability of startup companies and predict profits based on various attributes.
Constraints:
•
Ensure thorough data preprocessing, including handling missing values, encoding categorical variables, and addressing multicollinearity, to prepare the dataset for multilinear regression analysis while maintaining data integrity and quality.
•
Employ robust model evaluation techniques to assess the performance and reliability of the multilinear regression model, considering metrics such as R-squared, adjusted R-squared, and mean squared error, to ensure accurate predictions and actionable insights.
Benefits/Impact:
•
The multilinear regression model provides valuable insights into the factors influencing startup profitability, enabling informed decision-making regarding resource allocation, business strategies, and investment opportunities.
•
By predicting profits based on different attributes, the analytics company can assist startup companies in optimizing their operations, identifying areas for improvement, and enhancing overall financial performance, leading to increased competitiveness, sustainable growth, and enhanced market positioning.

'''







import pandas as pd
import numpy as np
import sidetable
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from feature_engine.outliers import Winsorizer
from sklearn.linear_model import LinearRegression
from statsmodels.tools.tools import add_constant
from sklearn.metrics import r2_score
#from statsmodels.stats.outliers_influence import inflation_variance_factor
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm
import joblib
import pickle
from sklearn.model_selection import GridSearchCV


In [2]:
startups = pd.read_csv('50_Startups.csv')
startups

R&D Spend  Administration  Marketing Spend       State     Profit
0   165349.20       136897.80        471784.10    New York  192261.83
1   162597.70       151377.59        443898.53  California  191792.06
2   153441.51       101145.55        407934.54     Florida  191050.39
3   144372.41       118671.85        383199.62    New York  182901.99
4   142107.34        91391.77        366168.42     Florida  166187.94
5   131876.90        99814.71        362861.36    New York  156991.12
6   134615.46       147198.87        127716.82  California  156122.51
7   130298.13       145530.06        323876.68     Florida  155752.60
8   120542.52       148718.95        311613.29    New York  152211.77
9   123334.88       108679.17        304981.62  California  149759.96
10  101913.08       110594.11        229160.95     Florida  146121.95
11  100671.96        91790.61        249744.55  California  144259.40
12   93863.75       127320.38        249839.44     Florida  141585.52
13   91992.39       135495.07        252664.93  California  134307.35
14  119943.24       156547.42        256512.92     Florida  132602.65
15  114523.61       122616.84        261776.23    New York  129917.04
16   78013.11       121597.55        264346.06  California  126992.93
17   94657.16       145077.58        282574.31    New York  125370.37
18   91749.16       114175.79        294919.57     Florida  124266.90
19   86419.70       153514.11             0.00    New York  122776.86
20   76253.86       113867.30        298664.47  California  118474.03
21   78389.47       153773.43        299737.29    New York  111313.02
22   73994.56       122782.75        303319.26     Florida  110352.25
23   67532.53       105751.03        304768.73     Florida  108733.99
24   77044.01        99281.34        140574.81    New York  108552.04
25   64664.71       139553.16        137962.62  California  107404.34
26   75328.87       144135.98        134050.07     Florida  105733.54
27   72107.60       127864.55        353183.81    New York  105008.31
28   66051.52       182645.56        118148.20     Florida  103282.38
29   65605.48       153032.06        107138.38    New York  101004.64
30   61994.48       115641.28         91131.24     Florida   99937.59
31   61136.38       152701.92         88218.23    New York   97483.56
32   63408.86       129219.61         46085.25  California   97427.84
33   55493.95       103057.49        214634.81     Florida   96778.92
34   46426.07       157693.92        210797.67  California   96712.80
35   46014.02        85047.44        205517.64    New York   96479.51
36   28663.76       127056.21        201126.82     Florida   90708.19
37   44069.95        51283.14        197029.42  California   89949.14
38   20229.59        65947.93        185265.10    New York   81229.06
39   38558.51        82982.09        174999.30  California   81005.76
40   28754.33       118546.05        172795.67  California   78239.91
41   27892.92        84710.77        164470.71     Florida   77798.83
42   23640.93        96189.63        148001.11  California   71498.49
43   15505.73       127382.30         35534.17    New York   69758.98
44   22177.74       154806.14         28334.72  California   65200.33
45    1000.23       124153.04          1903.93    New York   64926.08
46    1315.46       115816.21        297114.46     Florida   49490.75
47       0.00       135426.92             0.00  California   42559.73
48     542.05        51743.15             0.00    New York   35673.41
49       0.00       116983.80         45173.06  California   14681.40

In [3]:
startups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [4]:
x = startups.drop(['State','Profit'], axis = 1)
y = startups['Profit']

In [5]:
startups.describe()

R&D Spend  Administration  Marketing Spend         Profit
count      50.000000       50.000000        50.000000      50.000000
mean    73721.615600   121344.639600    211025.097800  112012.639200
std     45902.256482    28017.802755    122290.310726   40306.180338
min         0.000000    51283.140000         0.000000   14681.400000
25%     39936.370000   103730.875000    129300.132500   90138.902500
50%     73051.080000   122699.795000    212716.240000  107978.190000
75%    101602.800000   144842.180000    299469.085000  139765.977500
max    165349.200000   182645.560000    471784.100000  192261.830000

In [6]:
##auto EDA
import sweetviz
startups_report = sweetviz.analyze([x, 'x'])
startups_report.show_html('startups.html')

                                             |                                             | [  0%]   00:00 ->…

Report startups.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [7]:
num_pipeline = Pipeline(steps = [('impute', SimpleImputer(strategy = 'mean'))])
num_preprocessor = ColumnTransformer(transformers=[('impute', num_pipeline, x.columns)])

In [8]:
#outlier analysis

In [12]:
impute_data = num_pipeline.fit_transform(x)
impute_df = pd.DataFrame(impute_data, columns = x.columns)
impute_df

R&D Spend  Administration  Marketing Spend
0   165349.20       136897.80        471784.10
1   162597.70       151377.59        443898.53
2   153441.51       101145.55        407934.54
3   144372.41       118671.85        383199.62
4   142107.34        91391.77        366168.42
5   131876.90        99814.71        362861.36
6   134615.46       147198.87        127716.82
7   130298.13       145530.06        323876.68
8   120542.52       148718.95        311613.29
9   123334.88       108679.17        304981.62
10  101913.08       110594.11        229160.95
11  100671.96        91790.61        249744.55
12   93863.75       127320.38        249839.44
13   91992.39       135495.07        252664.93
14  119943.24       156547.42        256512.92
15  114523.61       122616.84        261776.23
16   78013.11       121597.55        264346.06
17   94657.16       145077.58        282574.31
18   91749.16       114175.79        294919.57
19   86419.70       153514.11             0.00
20   76253.86       113867.30        298664.47
21   78389.47       153773.43        299737.29
22   73994.56       122782.75        303319.26
23   67532.53       105751.03        304768.73
24   77044.01        99281.34        140574.81
25   64664.71       139553.16        137962.62
26   75328.87       144135.98        134050.07
27   72107.60       127864.55        353183.81
28   66051.52       182645.56        118148.20
29   65605.48       153032.06        107138.38
30   61994.48       115641.28         91131.24
31   61136.38       152701.92         88218.23
32   63408.86       129219.61         46085.25
33   55493.95       103057.49        214634.81
34   46426.07       157693.92        210797.67
35   46014.02        85047.44        205517.64
36   28663.76       127056.21        201126.82
37   44069.95        51283.14        197029.42
38   20229.59        65947.93        185265.10
39   38558.51        82982.09        174999.30
40   28754.33       118546.05        172795.67
41   27892.92        84710.77        164470.71
42   23640.93        96189.63        148001.11
43   15505.73       127382.30         35534.17
44   22177.74       154806.14         28334.72
45    1000.23       124153.04          1903.93
46    1315.46       115816.21        297114.46
47       0.00       135426.92             0.00
48     542.05        51743.15             0.00
49       0.00       116983.80         45173.06

In [14]:
winsor = Winsorizer(capping_method = 'iqr',
                   tail = 'both',
                   fold = 1.5,
                   variables = list(impute_df.columns))
outlier_pipeline = Pipeline([('wins', winsor)])

In [15]:
outlier_data = outlier_pipeline.fit_transform(impute_df)


In [16]:
scale_pipeline = Pipeline(steps = [('scale', MinMaxScaler())])
scale_preprocessor = ColumnTransformer(transformers=[('scale', scale_pipeline, outlier_data.columns)])

In [17]:
scale_data = scale_preprocessor.fit_transform(outlier_data)
scale_df = pd.DataFrame(scale_data, columns = x.columns)
scale_df

R&D Spend  Administration  Marketing Spend
0    1.000000        0.651744         1.000000
1    0.983359        0.761972         0.940893
2    0.927985        0.379579         0.864664
3    0.873136        0.512998         0.812235
4    0.859438        0.305328         0.776136
5    0.797566        0.369448         0.769126
6    0.814128        0.730161         0.270710
7    0.788018        0.717457         0.686493
8    0.729018        0.741733         0.660500
9    0.745906        0.436929         0.646443
10   0.616351        0.451506         0.485733
11   0.608845        0.308364         0.529362
12   0.567670        0.578836         0.529563
13   0.556352        0.641066         0.535552
14   0.725394        0.801327         0.543708
15   0.692617        0.543030         0.554864
16   0.471808        0.535270         0.560312
17   0.572468        0.714013         0.598948
18   0.554881        0.478772         0.625116
19   0.522650        0.778236         0.000000
20   0.461169        0.476424         0.633053
21   0.474084        0.780210         0.635327
22   0.447505        0.544293         0.642920
23   0.408424        0.414638         0.645992
24   0.465947        0.365388         0.297964
25   0.391080        0.671958         0.292427
26   0.455574        0.706845         0.284134
27   0.436093        0.582978         0.748613
28   0.399467        1.000000         0.250429
29   0.396769        0.774566         0.227092
30   0.374931        0.489928         0.193163
31   0.369741        0.772053         0.186989
32   0.383485        0.593294         0.097683
33   0.335617        0.394134         0.454943
34   0.280776        0.810055         0.446810
35   0.278284        0.257032         0.435618
36   0.173353        0.576825         0.426311
37   0.266527        0.000000         0.417626
38   0.122345        0.111636         0.392690
39   0.233194        0.241309         0.370931
40   0.173901        0.512041         0.366260
41   0.168691        0.254469         0.348614
42   0.142976        0.341852         0.313705
43   0.093776        0.579307         0.075319
44   0.134127        0.788072         0.060059
45   0.006049        0.554724         0.004036
46   0.007956        0.491260         0.629768
47   0.000000        0.640547         0.000000
48   0.003278        0.003502         0.000000
49   0.000000        0.500148         0.095749

In [18]:
joblib.dump(impute_data, 'lasso_impute_assignment')
joblib.dump(outlier_data, 'outlier_lasso_assignment')
joblib.dump(scale_data, 'scale_data_lasso_assignment')

['scale_data_lasso_assignment']

In [19]:
#model building

In [20]:
p = add_constant(scale_df)

In [22]:
basemodel = sm.OLS(y, p).fit()
basemodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Sat, 17 Feb 2024   Prob (F-statistic):           4.53e-30
Time:                        14:15:00   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            4.875e+04   4325.266     11.270      0.000       4e+04    5.75e+04
R&D Spend        1.332e+05   7465.065     17.846      0.000    1.18e+05    1.48e+05
Administration  -3522.6105   6703.264     -0.526      0.602    -1.7e+04    9970.360
Marketing Spend  1.285e+04   7761.431      1.655      0.105   -2777.182    2.85e+04
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                         10.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [25]:
vif = pd.Series([variance_inflation_factor(p.values, i)for i in range(p.shape[1])], index = p.columns)

In [26]:
vif

const              10.974190
R&D Spend           2.468903
Administration      1.175091
Marketing Spend     2.326773
dtype: float64

In [27]:
basemodel2 = sm.OLS(y, scale_df).fit()
basemodel2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Profit   R-squared (uncentered):                   0.979
Model:                            OLS   Adj. R-squared (uncentered):              0.978
Method:                 Least Squares   F-statistic:                              735.3
Date:                Sat, 17 Feb 2024   Prob (F-statistic):                    1.76e-39
Time:                        14:28:49   Log-Likelihood:                         -558.50
No. Observations:                  50   AIC:                                      1123.
Df Residuals:                      47   BIC:                                      1129.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
R&D Spend        1.202e+05   1.42e+04      8.496      0.000    9.18e+04    1.49e+05
Administration   5.535e+04   8060.904      6.866      0.000    3.91e+04    7.16e+04
Marketing Spend  5.466e+04   1.31e+04      4.179      0.000    2.83e+04     8.1e+04
==============================================================================
Omnibus:                        1.503   Durbin-Watson:                   1.242
Prob(Omnibus):                  0.472   Jarque-Bera (JB):                1.104
Skew:                           0.064   Prob(JB):                        0.576
Kurtosis:                       2.283   Cond. No.                         6.46
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
# Regularization Techniques: LASSO, RIDGE and ElasticNet Regression
################
### LASSO MODEL ###
from sklearn.linear_model import Lasso

In [35]:
lasso = Lasso(alpha = 0.13)

In [36]:
lasso.fit(scale_df, y)

Lasso(alpha=0.13)

In [37]:
lasso.intercept_

48746.7302450807

In [38]:
lasso.coef_

array([133228.15014663,  -3521.21312168,  12840.87448624])

In [39]:
pred = lasso.predict(scale_df)

In [40]:
# Adjusted r-square

s1 = lasso.score(scale_df, y)
s1

0.9507459934219078

In [41]:
# RMSE
np.sqrt(np.mean((pred - np.array(y))**2))

8855.344547124561

In [42]:
### RIDGE REGRESSION ###
from sklearn.linear_model import Ridge

In [43]:
rm = Ridge(alpha = 0.13)

In [44]:
rm.fit(scale_df, y)

Ridge(alpha=0.13)

In [45]:
rm.intercept_

48265.914426134914

In [46]:
rm.coef_

array([123968.39392632,   -334.21524258,  19345.68219103])

In [48]:
result = rm.coef_.flatten()
result

array([123968.39392632,   -334.21524258,  19345.68219103])

In [49]:
rm.alpha

0.13

In [50]:
rm_pred = rm.predict(scale_df)

In [51]:
# Adjusted r-square
s2 = rm.score(scale_df, y)
s2

0.9490757504105636

In [52]:
np.sqrt( np.mean((rm_pred - np.array(y))**2))

9004.238721312437

In [53]:

### ELASTIC NET REGRESSION ###
from sklearn.linear_model import ElasticNet

In [54]:
el = ElasticNet(alpha = 0.13)

In [55]:
el.fit(scale_df, y)

ElasticNet(alpha=0.13)

In [56]:
el.intercept_

64546.51253963901

In [57]:
el.coef_

array([62717.39578781,  7938.1909854 , 34137.73557891])

In [58]:
el_pred = el.predict(scale_df)

In [59]:
# Adjusted r-square
s3 = el.score(scale_df, y)
s3

0.8035991464706803

In [60]:
# RMSE
np.sqrt(np.mean(((el_pred - np.array(y))**2)))

17683.017101551635

In [61]:
#hyperparameter tuneing

In [62]:
lasso1 = Lasso()

In [63]:
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 0.13, 1, 5 ,10, 20]}

In [64]:
lasso_reg = GridSearchCV(lasso1, parameters, scoring = 'r2', cv = 5)

In [65]:
lasso_reg.fit(scale_df, y)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.769e+09, tolerance: 3.541e+06
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.373e+09, tolerance: 7.353e+06
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.821e+09, toleranc

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01,
                                   0.13, 1, 5, 10, 20]},
             scoring='r2')

In [66]:
lasso_reg.best_params_

{'alpha': 0.01}

In [67]:
lasso_reg.best_score_

-0.12163532060464262

In [70]:
lasso_reg_pred = lasso_reg.predict(scale_df)

In [71]:
# Adjusted r-square#
s4 = lasso_reg.score(scale_df, y)
s4

0.9507459940644803

In [72]:
#RMSE

In [73]:
np.sqrt(np.mean((lasso_reg_pred - np.array(y))**2))

8855.34448936072

In [74]:
# Ridge Regression


In [75]:
parameters1 = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 0.13, 1, 5 ,10, 20]}

In [76]:
ridge1 = Ridge()

In [77]:
ridge_reg = GridSearchCV(ridge1, parameters1, scoring = 'r2', cv = 5)

In [78]:
ridge_reg.fit(scale_df, y)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01,
                                   0.13, 1, 5, 10, 20]},
             scoring='r2')

In [79]:
ridge_reg.best_params_

{'alpha': 1e-15}

In [80]:
ridge_reg.best_score_

-0.12163534482007236

In [81]:
ridge1_pred = ridge_reg.predict(scale_df)

In [82]:
s5 = ridge_reg.score(scale_df, y)
s5

0.9507459940683246

In [83]:
# RMSE
np.sqrt(np.mean((ridge1_pred - np.array(y))**2))

8855.34448901514

In [84]:
# ElasticNet Regression


In [85]:
el1 = ElasticNet()

In [86]:
parameters2 = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 0.13, 1, 5 ,10, 20]}

In [87]:
el_reg = GridSearchCV(el1, parameters2, scoring = 'r2', cv = 5)

In [88]:
el_reg.fit(scale_df, y)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.807e+09, tolerance: 3.541e+06
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.376e+09, tolerance: 7.353e+06
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.830e+09, toleranc

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01,
                                   0.13, 1, 5, 10, 20]},
             scoring='r2')

In [89]:
el_reg.best_params_

{'alpha': 1e-15}

In [90]:

el_reg.best_score_

-0.12163534482007705

In [91]:
el_reg_pred = el_reg.predict(scale_df)

In [92]:
# Adjusted r-square
s6 = el_reg.score(scale_df, y)
s6

0.9507459940683246

In [93]:
#RMSE

In [95]:
np.sqrt(np.mean((el_reg_pred - np.array(y))**2))

8855.344489015139

In [97]:
scores_all = pd.DataFrame({'model': ['Lasso','Ridge','ElasticNet','lasso_reg','ridge_reg','el_reg'], 'scores': [s1,s2,s3,s4,s5,s6]})

In [98]:
scores_all

model    scores
0       Lasso  0.950746
1       Ridge  0.949076
2  ElasticNet  0.803599
3   lasso_reg  0.950746
4   ridge_reg  0.950746
5      el_reg  0.950746

In [99]:
final = el_reg.best_estimator_
final

ElasticNet(alpha=1e-15)

In [100]:
#save the model

In [102]:
pickle.dump(final, open('Elastic_model_assignment', 'wb'))

In [103]:
#prediction

In [114]:
test = pd.read_csv('50_Startups.csv')
test1 = test.drop(['State', 'Profit'], axis = 1)

In [117]:
impute1 = SimpleImputer(strategy = 'mean')

winsor1 = Winsorizer(capping_method = 'iqr', tail = 'both', variables = list(test1.columns))
scale = MinMaxScaler()
impute_d = impute1.fit_transform(test1)
impute_f = pd.DataFrame(impute_d, columns = test1.columns)
outlier_d = winsor1.fit_transform(impute_f)
scale_d = scale.fit_transform(outlier_d)

In [118]:
scale_f = pd.DataFrame(scale_d, columns = test1.columns)
model_elastic = joblib.load(open('Elastic_model_assignment','rb'))

In [122]:
prediction = pd.DataFrame(model_elastic.predict(scale_f), columns = ['Profit'])


In [123]:
final_prediction = pd.concat([test1, prediction], axis = 1)
final_prediction

R&D Spend  Administration  Marketing Spend         Profit
0   165349.20       136897.80        471784.10  192521.252890
1   162597.70       151377.59        443898.53  189156.768232
2   153441.51       101145.55        407934.54  182147.279096
3   144372.41       118671.85        383199.62  173696.700026
4   142107.34        91391.77        366168.42  172139.514183
5   131876.90        99814.71        362861.36  163580.780571
6   134615.46       147198.87        127716.82  158114.096669
7   130298.13       145530.06        323876.68  160021.363048
8   120542.52       148718.95        311613.29  151741.699699
9   123334.88       108679.17        304981.62  154884.684110
10  101913.08       110594.11        229160.95  135509.016367
11  100671.96        91790.61        249744.55  135573.712961
12   93863.75       127320.38        249839.44  129138.054182
13   91992.39       135495.07        252664.93  127487.991663
14  119943.24       156547.42        256512.92  149548.646335
15  114523.61       122616.84        261776.23  146235.159985
16   78013.11       121597.55        264346.06  116915.405401
17   94657.16       145077.58        282574.31  130192.447208
18   91749.16       114175.79        294919.57  129014.226806
19   86419.70       153514.11             0.00  115635.216367
20   76253.86       113867.30        298664.47  116639.669231
21   78389.47       153773.43        299737.29  117319.451640
22   73994.56       122782.75        303319.26  114706.981717
23   67532.53       105751.03        304768.73  109996.615221
24   77044.01        99281.34        140574.81  113362.966113
25   64664.71       139553.16        137962.62  102237.725065
26   75328.87       144135.98        134050.07  110600.575350
27   72107.60       127864.55        353183.81  114408.071457
28   66051.52       182645.56        118148.20  101660.026005
29   65605.48       153032.06        107138.38  101794.983452
30   61994.48       115641.28         91131.24   99452.372936
31   61136.38       152701.92         88218.23   97687.856276
32   63408.86       129219.61         46085.25   99001.328985
33   55493.95       103057.49        214634.81   97915.007805
34   46426.07       157693.92        210797.67   89039.273741
35   46014.02        85047.44        205517.64   90511.599568
36   28663.76       127056.21        201126.82   75286.174585
37   44069.95        51283.14        197029.42   89619.537708
38   20229.59        65947.93        185265.10   69697.430648
39   38558.51        82982.09        174999.30   83729.011977
40   28754.33       118546.05        172795.67   74815.953991
41   27892.92        84710.77        164470.71   74802.556239
42   23640.93        96189.63        148001.11   70620.411821
43   15505.73       127382.30         35534.17   60167.039963
44   22177.74       154806.14         28334.72   64611.354916
45    1000.23       124153.04          1903.93   47650.649687
46    1315.46       115816.21        297114.46   56166.206853
47       0.00       135426.92             0.00   46490.588983
48     542.05        51743.15             0.00   49171.388158
49       0.00       116983.80         45173.06   48215.134111